# Launch Refinery Margins Viewer

This notebook launches the **refinery_by_country** display at http://localhost:8763.

**Use this to quickly test the refinery margins viewer.**

## Display Info

- **Name**: refinery_by_country
- **Data**: Refinery capacity by country (2006-2021)
- **Panels**: 10 time series plots (one per country)
- **Port**: 8763

In [ ]:
import subprocess
import time
import webbrowser
from pathlib import Path
import os
import signal

print("Refinery Margins Viewer Launcher")
print("="*70)

## 1. Check Display Exists

In [ ]:
# Display location
display_dir = Path("output/refinery_by_country")

print(f"\nChecking display at: {display_dir.absolute()}")
print("-"*70)

if not display_dir.exists():
    print(f"\n❌ ERROR: Display directory not found!")
    print(f"\nThe display needs to be created first.")
    print(f"Run notebook: 13_refinery_margins_demo.ipynb")
else:
    # Check required files
    required_files = [
        display_dir / "index.html",
        display_dir / "config.json",
        display_dir / "displays" / "displayList.json",
    ]
    
    all_exist = True
    for file_path in required_files:
        exists = file_path.exists()
        status = "✓" if exists else "✗"
        print(f"  {status} {file_path.relative_to(display_dir)}")
        if not exists:
            all_exist = False
    
    # Check panels
    panels_dir = display_dir / "displays" / "refinery_by_country" / "panels"
    panel_count = len(list(panels_dir.glob("*.png"))) if panels_dir.exists() else 0
    print(f"  ✓ {panel_count} panels")
    
    if all_exist and panel_count > 0:
        print("\n✅ Display is ready!")
    else:
        print("\n⚠️  Some files may be missing. Run 13_refinery_margins_demo.ipynb first.")

## 2. Kill Existing Server

In [ ]:
port = 8763

print(f"\nChecking port {port}...")
print("-"*70)

try:
    result = subprocess.run(
        ["lsof", "-ti", f":{port}"],
        capture_output=True,
        text=True
    )
    
    if result.stdout.strip():
        pids = result.stdout.strip().split()
        print(f"⚠️  Port {port} is in use by process(es): {', '.join(pids)}")
        
        for pid in pids:
            try:
                os.kill(int(pid), signal.SIGTERM)
                print(f"  ✓ Killed process {pid}")
            except (ProcessLookupError, PermissionError) as e:
                print(f"  ℹ️  {e}")
        
        time.sleep(1)
        print(f"\n✅ Port {port} cleared")
    else:
        print(f"✅ Port {port} is available")
except Exception as e:
    print(f"ℹ️  Could not check port: {e}")

## 3. Start HTTP Server

In [ ]:
print(f"\nStarting HTTP server...")
print("-"*70)

# CRITICAL: Start from display root directory
server_process = subprocess.Popen(
    ["python3", "-m", "http.server", str(port)],
    cwd=display_dir,  # ← This is critical!
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

print(f"✓ Server started (PID: {server_process.pid})")
print(f"  Serving from: {display_dir.absolute()}")
print(f"  Port: {port}")

# Wait for server to start
time.sleep(2)

if server_process.poll() is None:
    print(f"\n✅ Server is running!")
else:
    print(f"\n❌ Server failed to start")

## 4. Open Browser

In [ ]:
url = f"http://localhost:{port}/"

print(f"\nOpening browser...")
print("-"*70)

try:
    webbrowser.open(url)
    print(f"✓ Browser opened to: {url}")
except Exception as e:
    print(f"⚠️  Could not open browser: {e}")
    print(f"\nManually open: {url}")

print(f"\n{'='*70}")
print(f"🌐 VIEWER URL: {url}")
print(f"{'='*70}")
print(f"\nExpected Result:")
print(f"  ✓ 10 time series plots (one per country)")
print(f"  ✓ Line charts showing refinery capacity over time")
print(f"  ✓ Filter by country dropdown")
print(f"  ✓ Sort by avg/max/min capacity")
print(f"  ✓ Default layout: 3 columns x 2 rows")
print(f"\nCountries:")
print(f"  Algeria, Denmark, Germany, Italy, Netherlands,")
print(f"  Norway, Romania, Russian Federation, Turkey, United Kingdom")
print(f"\nServer PID: {server_process.pid}")

## 5. Server Management

### Stop Server

In [ ]:
# Stop the server
try:
    if 'server_process' in globals() and server_process.poll() is None:
        server_process.terminate()
        server_process.wait(timeout=5)
        print(f"✓ Server stopped (PID: {server_process.pid})")
    else:
        print("ℹ️  Server is not running")
except Exception as e:
    print(f"⚠️  Error stopping server: {e}")

### Check Server Status

In [ ]:
# Check if server is still running
if 'server_process' in globals():
    if server_process.poll() is None:
        print(f"✓ Server is running (PID: {server_process.pid})")
        print(f"  URL: http://localhost:{port}/")
    else:
        print(f"✗ Server has stopped")
else:
    print("ℹ️  Server has not been started yet")

## Quick Reference

### Data Source
- **File**: `_data/refinery_margins.csv`
- **Rows**: 1,920 (192 months per country)
- **Countries**: 10
- **Date range**: 2006-01-01 to 2021-12-01

### Display Stats
- **Name**: refinery_by_country
- **Description**: Refinery Capacity by Country - Time Series
- **Panels**: 10 (one per country)
- **Meta variables**: country, avg_capacity, max_capacity, min_capacity, n_obs
- **Layout**: 3 columns x 2 rows

### Working Viewers

| Port | Display | Status |
|------|---------|--------|
| 9000 | simple_static | ✅ Demo |
| 8762 | notebook_demo | ✅ Demo |
| **8763** | **refinery_by_country** | ✅ **This viewer** |

### Manual Server (Alternative)

```bash
cd examples/output/refinery_by_country
python3 -m http.server 8763
```

Then open: http://localhost:8763/